In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
from cassandra.cluster import Cluster
from os.path import expanduser
home = expanduser("~")
with open(home+'/.aws/cassandra_cluster') as f:
    cluster_ip = f.readline().strip('\n')
cluster = Cluster([cluster_ip])
session = cluster.connect('measurements')

In [ ]:
from solardatatools import standardize_time_axis, make_2d, plot_2d, fix_time_shifts
from statistical_clear_sky.utilities.data_conversion import make_time_series

In [ ]:
import time

#### Measuing the execution time for three sites 1

In [ ]:
execution_start_time = time.time()

cql = """
    select site, meas_name, ts, sensor, meas_val_f 
    from measurement_raw
    where site in ('TADBC1078041', 'ZT163185000441C2053', 'TADBC1078234')
        and meas_name = 'ac_power';
"""
rows = session.execute(cql)

execution_end_time = time.time()

print("Query execution time: %s" % (execution_end_time - execution_start_time))

In [ ]:
data_frame = pd.DataFrame(list(rows))
data_frame.replace(-999999.0, np.NaN, inplace=True)

In [ ]:
time_series_data_frame = make_time_series(data_frame, return_keys=False)
time_series_data_frame

In [ ]:
time_series_data_frame.plot()

In [ ]:
standardized_data_frame = standardize_time_axis(time_series_data_frame)
standardized_data_frame

In [ ]:
standardized_data_frame.plot()

In [ ]:
power_matrix_1 = make_2d(standardized_data_frame, key='ac_power_01', zero_nighttime=True, interp_missing=True)
power_matrix_1

In [ ]:
plot_2d(power_matrix_1)